In [1]:
from mlx_lm import load, generate
import pandas as pd

In [2]:
# %%bash
# python -m mlx_lm.convert --hf-path jphme/em_german_leo_mistral --mlx-path model/ -q 

In [3]:
# TODO: Choose one
inter_df = pd.read_csv("test.csv")
#inter_df = pd.read_csv("test2.csv")

In [4]:
inter_df.head()

,patientid,sex,age,ANA,EXA,DIA_text,DIA_code,OP_text,OP_code,EXA_formatted
0,61.0,male,32.0,Beschwerden re. Achillessehne. NMR unauffällig...,Re. Achillessehne: Verdickung im Bereich der A...,Retropatellare Chondromalazie li; Insertionste...,M22.4 G,NaN,NaN,rechts leiste druckschmerz am os pubis zum os ...
1,67.0,male,41.0,Seit mehreren Monaten Beschwerden re. Achilles...,Re. Achillessehne: Deutliche Auftreibung der A...,V.a.laterale Chondromalazie bei st.n. Außenmen...,M23.3- V,NaN,NaN,massiver druckschmerz dort
2,72.0,male,19.0,. .Gestern beim Training stechende Schmerzen l...,Klinischer Befund des Oberschenkels li.deutl.V...,Z.n.arthroskopischer Chirurgie des Kniegelenkes.,M23.3- Z,NaN,NaN,NaN
3,72.0,male,23.0,beide Knie Schmerzen bei Treppe steigen oder L...,bd. Knie: frei Bewegl.; Patella o.B.; keine MZ...,Z.n.IM-Teilresektion re.,S83.2 Z,NaN,NaN,NaN
4,84.0,male,57.0,(08.19 Uhr); bewegungsabhängig.Schmerzen wiede...,Klinischer Befund des Kniegelenks li.: Drucksc...,Beginnende Gonarthrose bds. li. mehr als re.,M17.9,NaN,NaN,klinischer befund des kniegelenks li druckschm...


In [5]:
model, tokenizer = load("model/")

In [6]:
few_shot_examples = [
    ("links knie diskr schwellung, kein anteromedialer druckschmerz med äußerechts Seite des Kniegelenks, kein druckschmerz kniescheiben, keine meniskuszeichen rechts knie keine schwellung, druckschmerz proximal seitlich kniescheibe", 
     "Mein linkes Knie hat keine Schwellung, und es gibt keine Schmerzen beim Drücken auf der äußerechts Seite des Kniegelenks und der Kniescheibe. Mein rechtes Knie hat auch keine Schwellung, aber in der Nähe der Seite der Kniescheibe, gibt es Schmerzen beim Drücken."),
    ("rechts knie deutlicher schwellung, keine überstreckung des knies, streckdefizit des knies, angemessene beugung des knies, keine rötung", 
     "Mein rechtes Knie hat eine Schwellung, ich kann es nicht überstrecken und auch nicht komplett strecken. Die Beugung des Knies ist nur angemessen möglich. Mein Knie ist nicht gerötet."),
]

# Construct the few-shot prompt
instruction = "Du bist ein Patient mit den folgenden Symptomen. Ersetze alle lateinischen und deutschen Fachbegriffe und formuliere es umgangssprachlich aus deiner Sicht."
few_shot_prompt = f"{instruction}\n\n" + "\n\n".join([f"USER: {inp}\nASSISTANT: {out}" for inp, out in few_shot_examples])

In [7]:
def generate_response(text):
    if (text != "") and (text is not None) and not (isinstance(text, float)):
        prompt = f"{few_shot_prompt}\n\nUSER: {text}\nASSISTANT:"
        generated_text = generate(model, tokenizer, prompt=prompt, max_tokens=200, verbose=False)
        return generated_text
    return ""

In [8]:
for idx, row in inter_df.head(10).iterrows():
    text = row['EXA_formatted']
    ouput_text = generate_response(text)
    inter_df.at[idx, 'EXA_interpreted'] = ouput_text

In [9]:
def remove_after_newline(text):
    return text.split('\n')[0] if isinstance(text, str) else text

In [10]:
inter_df["EXA_interpreted"] = inter_df["EXA_interpreted"].apply(remove_after_newline)

In [11]:
inter_df.to_csv("interpreted_data_mlx.csv")